In [ ]:
from keras.utils.generic_utils import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json

from sklearn.metrics import accuracy_score

In [ ]:
start = datetime(2012,1,1)
end = datetime(2022,1,1)

In [ ]:
qb = QuantBook()

In [ ]:
symbol = qb.AddCrypto("BTCUSD",Resolution.Daily).Symbol
history = qb.History(symbol, start, end).loc[symbol]

In [ ]:
daily_pct_change = history[['open','high','low','close','volume']].pct_change().dropna()
df = daily_pct_change

In [ ]:
indexes = df[((df.volume == float('inf')))].index

for i in indexes:
    df.at[i,'volume'] = max(df.volume.drop(indexes))

In [ ]:
n_steps = 30
features = []
labels = []

for i in range(len(df) - n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df['close'].iloc[i+n_steps] >= 0:
        label = 1
    else:
        label = 0
    labels.append(label)

In [ ]:
features = np.array(features)
labels = np.array(labels)

In [ ]:
train_length = int(len(features)*0.7)

X_train = features[:train_length]
X_test = features[train_length:]
Y_train = labels[:train_length]
Y_test = labels[train_length:]

In [ ]:
model = Sequential([
    Dense(30,input_shape=X_train[0].shape,activation = 'relu'),
    Dense(20,activation = 'relu'),
    Flatten(),
    Dense(1, activation = 'sigmoid')])

In [ ]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy','mse']
)

In [ ]:
model.fit(X_train, Y_train, epochs=50)

In [ ]:
y_hat = model.predict(X_test)
Threshold = 0.5
y_pred = np.where(y_hat > Threshold, 1, 0)

In [ ]:
accuracy_score(Y_test,y_pred)